# Bibliotecas

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import funcoes_modelos as fmod
import funcoes_bow as fbow
import funcoes_cos as fcos

import time

In [ ]:
tf.keras.backend.set_floatx('float16')

## Configurações

In [ ]:
# flag_cpu = True
# if flag_cpu == True:
#     # force CPU (make CPU visible)
#     cpus = tf.config.experimental.list_physical_devices('CPU')
#     print(cpus)
#     tf.config.set_visible_devices([], 'GPU')  # hide the GPU
#     tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
#     tf.config.get_visible_devices()

## Constantes

In [ ]:
SEED = 42

# Leitura dos arquivos

In [ ]:
#arquivos = ['hn_balanceado']
#arquivos = ['hn_desbalanceado']
#arquivos = ['sn_balanceado']
#arquivos = ['sn_desbalanceado']

In [ ]:
arquivos = ['hn_balanceado', 'hn_desbalanceado', 'sn_balanceado', 'sn_desbalanceado']

lista_df = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/{arquivo}.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df.append(df)

In [ ]:
#lista_df[0].dtypes

In [ ]:
#print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {lista_df[0].shape[0]}\t| {lista_df[2].shape[0]}\t|\nDesbalanceado\t| {lista_df[1].shape[0]}\t| {lista_df[3].shape[0]}\t|")

# Separando o Dataset

In [ ]:
def train_test_valid(df):
    
    X = df[["titulo_1", "titulo_2"]]
    y = df["match"].to_list()
    
    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.285, random_state = SEED, stratify = y_train_valid)
    
    return X_train, y_train, X_test, y_test, X_valid, y_valid


In [ ]:
lista_train_test_valid = []
for df in lista_df:
    lista_train_test_valid.append(train_test_valid(df))

In [ ]:
#for nome, dataset in zip(arquivos, lista_train_test_valid):    
#    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

# Experimentos Com Os Modelos

## Funções

In [ ]:
def salvar_df_resultado(report, modelo, nome_dataset, tempo_exec):
    
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = modelo
    df_resultado['dataset'] = nome_dataset
    df_resultado['tempo'] = tempo_exec

    df_resultado.to_csv(f'Dados/Resultados/{modelo}/{nome_dataset}_resultado.csv', index = True)
    
    return df_resultado


def salvar_modelo(modelo, dataset, metodo):
    
    nome_arquivo = f"Dados/Modelos/modelo_{dataset}_{metodo}"
    pickle.dump(modelo, open(nome_arquivo, 'wb'))

def carregar_modelo(nome, metodo):
    
    modelo = pickle.load(open(f"Dados/Modelos/modelo_{nome}_{metodo}", 'rb'))
    return modelo


def salvar_modelo_bert(modelo, dataset, metodo):
    
    nome_arquivo = f"Dados/Modelos/{metodo}/{dataset}/"
    modelo.save_pretrained(nome_arquivo)

def carregar_modelo_bert(nome, dataset, metodo):
    
    from transformers import TFAutoModel
    
    modelo = TFAutoModel.from_pretrained(f"Dados/Modelos/{metodo}/{dataset}/")
    return modelo

## BERT, roBERTa, XLMR e ELECTRA

In [ ]:
#metodos = ("BERT", "roBERTa")
#metodos = ("roBERTa",  "XLMR")
#metodos = ("XLMR", "ELECTRA")
#metodos = ("ELECTRA", "BERT")
metodos = ("a", "b")

In [ ]:
#metodos = ("BERT", "roBERTa",  "XLMR", "ELECTRA")

In [ ]:
#fim = 70
'''for metodo in metodos:

    lista_df_resultado = []
    
    for nome, dataset in zip(arquivos, lista_train_test_valid):    
        
        X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

        start_time = time.time()
        (nome, historico, y_test, y_pred, modelo) = fmod.pipeline_bert(metodo, nome, X_train, y_train, X_valid, y_valid, X_test, y_test)
        #(nome, historico, y_test, y_pred, modelo) = fmod.pipeline_bert(metodo, nome, X_train[:fim], y_train[:fim], X_valid[:fim], y_valid[:fim], X_test[:fim], y_test[:fim])
        end_time = time.time()
        runtime = end_time - start_time
        
        pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/{metodo}/{nome}_historico.csv', index = False)

        report = classification_report(y_test, y_pred, output_dict = True)
        
        df_resultado = salvar_df_resultado(report, metodo, nome, runtime)
        salvar_modelo_bert(modelo, nome, metodo)

        lista_df_resultado.append(df_resultado)'''

## BoW Co ocorrencia

In [ ]:
metodo = "BoW_rf"
lista_df_resultado = []
#fim = 150

for nome, dataset in zip(arquivos, lista_train_test_valid):
    
    #print(f"\n\nNome: {nome}\n\n")
    
    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

    start_time = time.time()
    
    name_dataset, y_test, y_pred, modelo = fbow.pipeline_rf(nome, X_train, y_train, X_valid, y_valid, X_test, y_test)
    # name_dataset, y_test, y_pred = fbow.pipeline_rf(nome, X_train[:fim], y_train[:fim], X_valid[:fim], y_valid[:fim], X_test[:fim], y_test[:fim])
    
    #print(f"\n\n{nome}\n\n")

    end_time = time.time()
    runtime = end_time - start_time

    report = classification_report(y_test, y_pred, output_dict = True)
    
    df_resultado = salvar_df_resultado(report, metodo, nome, runtime)
    salvar_modelo(modelo, nome, "BoW_rf")

    lista_df_resultado.append(df_resultado)

## Similaridade de Cosseno

In [ ]:
'''metodo = "Cos"
lista_df_resultado = []
thresh_list = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for nome, dataset in zip(arquivos, lista_train_test_valid):

    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

    for threshold in thresh_list:
        start_time = time.time()

        name_dataset, y_test, y_pred = fcos.cos_threshold(nome, X_test, y_test, threshold)

        end_time = time.time()
        runtime = end_time - start_time

        report = classification_report(y_test, y_pred, output_dict = True)
        
        df_resultado = salvar_df_resultado(report, f'{metodo}_0{int(threshold*10)}', nome, runtime)

        lista_df_resultado.append(df_resultado)'''

In [ ]:
#len(lista_df_resultado)